In [1]:
import mne
from mne.datasets import sample
from mne import io
from mne.preprocessing import create_ecg_epochs, create_eog_epochs
from mne.preprocessing import ICA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
from os import path
import fnmatch


In [2]:
#test=mne.io.read_raw_bdf('/home/dcellier/RDSS/AlphaStudy_Data/eegData/eeg_eyetracking/extra_or_defunct/sub_212_eyetracking_testing.bdf',montage=mne.channels.read_montage('biosemi64'),preload=True)

In [3]:
#test.plot()

In [4]:
ROOT_eeg='/home/dcellier/RDSS/AlphaStudy_Data/eegData/eeg_raw/'
ROOT_behav='/home/dcellier/RDSS/AlphaStudy_Data/eegData/eeg_behavior_data/'
ROOT_eyetrack='/data/backed_up/shared/AlphaStudy_data/placeholders/eyetracking_compiled_trials/'

In [62]:
testing_sub=mne.io.read_raw_bdf(ROOT_eeg+'sub_233_alpha_placeholders_01_23_20.bdf',montage=mne.channels.read_montage('biosemi64'),preload=True)

Extracting EDF parameters from /home/dcellier/RDSS/AlphaStudy_Data/eegData/eeg_raw/sub_233_alpha_placeholders_01_23_20.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2202111  =      0.000 ...  4300.998 secs...


<ipython-input-62-340d72ad9b91>:1: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8']. Their position has been left untouched.
  testing_sub=mne.io.read_raw_bdf(ROOT_eeg+'sub_233_alpha_placeholders_01_23_20.bdf',montage=mne.channels.read_montage('biosemi64'),preload=True)


In [63]:
testing_sub.info

<Info | 17 non-empty fields
    bads : list | 0 items
    ch_names : list | Fp1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, ...
    chs : list | 73 items (EEG: 72, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : Transform | 3 items
    dig : list | 67 items (3 Cardinal, 64 EEG)
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 104.0 Hz
    meas_date : tuple | 2020-01-23 09:59:13 GMT
    nchan : int | 73
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 512.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject_inf

In [64]:
behav_files=pd.DataFrame()
all_files=[]
for f in os.listdir(ROOT_behav):
    if '233_' in f:
        print(f)
        behav_file=pd.read_csv(ROOT_behav+f,engine='python')
        all_files.append(behav_file)
        behav_files=behav_files.append(behav_file,ignore_index=True)
#behav_files.dropna(axis=0,inplace=True,how='any')
#print(behav_files)

233_alpha_placeholders_matchingBrad_01_1_2020_Jan_23_0957.csv
233_alpha_placeholders_matchingBrad_01_2_2020_Jan_23_1007.csv
233_alpha_placeholders_matchingBrad_01_3_2020_Jan_23_1014.csv
233_alpha_placeholders_matchingBrad_01_4_2020_Jan_23_1021.csv
233_alpha_placeholders_matchingBrad_01_5_2020_Jan_23_1029.csv
233_alpha_placeholders_matchingBrad_01_6_2020_Jan_23_1035.csv
233_alpha_placeholders_matchingBrad_01_7_2020_Jan_23_1042.csv
233_alpha_placeholders_matchingBrad_01_9_2020_Jan_23_1058.csv
233_alpha_placeholders_matchingBrad_01_10_2020_Jan_23_1105.csv
233_alpha_placeholders_matchingBrad_01_8_2020_Jan_23_1051.csv
compiledAllBlocks_fromPreprocScript_s233_before_eyes.csv
compiledAllBlocks_fromPreprocScript_s233_after_eyes.csv


/home/dcellier/condaenvs/p3env/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [65]:
behav_files.columns

Index(['Eyetrack_trial', 'ITI', 'RT', 'SOA', 'Tar,Dis,Other',
       'Timestamp_start_cue', 'Timestamp_start_mask', 'Timestamp_start_probe',
       'Timestamp_stop_cue', 'Timestamp_stop_mask', 'Timestamp_stop_probe',
       'TrialTrigs(m,c,p)', 'Unnamed: 0', 'Unnamed: 0.1', 'block', 'corrResp',
       'dis_color', 'leftX_cue', 'leftX_mask', 'leftX_probe', 'leftY_cue',
       'leftY_mask', 'leftY_probe', 'rightX_cue', 'rightX_mask',
       'rightX_probe', 'rightY_cue', 'rightY_mask', 'rightY_probe', 'subResp',
       'tar,dis stim_loc', 'tar_color', 'trialCorr?', 'trialNum',
       'trial_trigs', 'trial_type', 'triggers'],
      dtype='object')

In [66]:
behav_files=behav_files.sort_values(['block','trialNum'])
#behav_files

In [67]:
np.unique(list(behav_files['block']))

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [68]:
behav_files.to_csv('/home/dcellier/RDSS/AlphaStudy_Data/eegData/eeg_behavior_data/compiledAllBlocks_s233.csv')

In [69]:
eyetrack_compiled_path='/data/backed_up/shared/AlphaStudy_data/placeholders/eyetracking_compiled_trials/'
sub_name='233'
pattern=sub_name+'_alpha_placeholders_matchingBrad_*_*_*_*_*_*.csv'
pattern2=sub_name+'_alpha_placeholders_matchingBrad_*_*_*_*_*.csv'
pattern3=sub_name+'_alpha_placeholders_matchingBrad_*_*_*_*_*_TRUNCATED_TO_MATCH_BDF.csv'
pattern4=sub_name+'_alpha_placeholders_matchingBrad_*_*_*_*_*_*_TRUNCATED_TO_MATCH_BDF.csv'
behav_files=pd.DataFrame()
for f in os.listdir(ROOT_behav):
    if fnmatch.fnmatch(f,pattern) or fnmatch.fnmatch(f,pattern2) or fnmatch.fnmatch(f,pattern3) or fnmatch.fnmatch(f,pattern4):
        print(f)
        behav_file=pd.read_csv(ROOT_behav+f,engine='python')
        behav_files=behav_files.append(behav_file,ignore_index=True)
behav_files_sorted=behav_files.sort_values(['block','trialNum']).reset_index()
eyes=pd.read_csv(eyetrack_compiled_path+'sub_'+sub_name+'_trialsCompiled.csv')
behav_files=pd.concat([behav_files_sorted,eyes],ignore_index=False,axis=1)

233_alpha_placeholders_matchingBrad_01_1_2020_Jan_23_0957.csv
233_alpha_placeholders_matchingBrad_01_2_2020_Jan_23_1007.csv
233_alpha_placeholders_matchingBrad_01_3_2020_Jan_23_1014.csv
233_alpha_placeholders_matchingBrad_01_4_2020_Jan_23_1021.csv
233_alpha_placeholders_matchingBrad_01_5_2020_Jan_23_1029.csv
233_alpha_placeholders_matchingBrad_01_6_2020_Jan_23_1035.csv
233_alpha_placeholders_matchingBrad_01_7_2020_Jan_23_1042.csv
233_alpha_placeholders_matchingBrad_01_9_2020_Jan_23_1058.csv
233_alpha_placeholders_matchingBrad_01_10_2020_Jan_23_1105.csv
233_alpha_placeholders_matchingBrad_01_8_2020_Jan_23_1051.csv


In [70]:
#behav_files_sorted

In [71]:
#behav_files

In [72]:
events=mne.find_events(testing_sub, initial_event=True,min_duration=(2/512))# 133 = start block flag, 135 = stop block

3022 events found
Event IDs: [  101   103   105   107   109   111   113   115   117   119   121   123
   125   127   129   131   133   135   225 65536 65737]


In [73]:
## Generating Epochs obj so I can make sure the metadata matches up
cue_event_id={'disShortCue_trig':101,'disLongCue_trig':103,'tarShortCue_trig':105,
             'tarLongCue_trig':107,'neutShortCue_trig':109, 'neutLongCue_trig':111}
sub224_cueEps=mne.Epochs(testing_sub, events=events, baseline=None, event_id=cue_event_id,
                         tmin=-1.3,tmax=1,metadata=behav_files)

600 matching events found
No baseline correction applied
Adding metadata with 37 columns
0 projection items activated


In [74]:
len(behav_files)

600

In [75]:
np.sum(sub224_cueEps['disLongCue_trig'].metadata['SOA']!='long')

0

In [60]:
len(events)

3020

In [61]:
## SEPERATING TRIGS BY BLOCKS
    # not all start_block trigs came through 
    # but all of the end block triggers did,
    # so we're basing off of those

blocks={}
block_counter=0
trig_counter=0
while block_counter<10: # we usually have 10 blocks of EEG
    ev=events[trig_counter]
    #print(trig_counter)

    if ev[2]==135: # then we've found the end of the block, move on to the next
        blocks[str(block_counter)].append(ev)
        block_counter += 1
    else:
        if str(block_counter) not in blocks.keys():
            blocks[str(block_counter)]=[]
        blocks[str(block_counter)].append(ev)
        
    trig_counter += 1
        

IndexError: index 3020 is out of bounds for axis 0 with size 3020

In [ ]:
for k in blocks.keys():
    probe_count=len([ev[2] for ev in blocks[k] if ev[2] in [113,115,117,119,121,123]])
    if probe_count < 60:
        print('Block {0} has {1} probe events'.format(k,str(probe_count)))

In [ ]:
ITI_evs=[ev[2] for ev in events if ev[2]==127]
mask_evs=[ev[2] for ev in events if ev[2]==125]
probe_evs=[ev[2] for ev in events if ev[2] in [113,115,117,119,121,123]]
cue_evs=[ev[2] for ev in events if ev[2] in [101,103,105,107,109,111]]
resp_evs=[ev[2] for ev in events if ev[2] in [129,131]]
Startblock_evs=[ev[2] for ev in events if ev[2] ==133]
Stopblock_evs=[ev[2] for ev in events if ev[2] ==135]
print("Len ITI trigs: "+ str(len(ITI_evs)))
print("Len Mask trigs: "+ str(len(mask_evs)))
print("Len Probe trigs: "+str(len(probe_evs)))
print("Len cue events: "+str(len(cue_evs)))
print("Len responses: "+str(len(resp_evs)))
print("Len start block trigs: "+str(len(Startblock_evs)))
print("Len stop block trigs: "+str(len(Stopblock_evs)))

In [ ]:
probe_block0=[ev[2] for ev in blocks['0'] if ev[2] in [113,115,117,119,121,123]]
probe_block0
enumerated_trigs=[]
for i,p in enumerate(probe_block0):
    enumerated_trigs.append([i,p])

In [ ]:
behav_trigs_probe_block0=list(behav_files[behav_files['block']==1]['trial_trigs'])
behav_trigs_probe_block0[0][5:10]
enumerated_behav=[]
for i,p in enumerate(behav_trigs_probe_block0):
    #print(i,int(p[5:9]))
    enumerated_behav.append([i,int(p[5:9])])


In [ ]:
for t,b in zip(enumerated_trigs,enumerated_behav):
    print(t,b)

In [ ]:
#cue_evs[:60]
#cue_evs[60:120]
#cue_evs[120:180]
#cue_evs[180:240]

In [ ]:
#all_trigs[:10]

In [ ]:
mask_trigs=[ev[2] for ev in events if ev[2]==125]
len(mask_trigs)

In [ ]:
all_trigs=[ev[2] for ev in events if ev[2] in [101,103,105,107,109,111,113,115,117,119,121,123]]

In [ ]:
len(all_trigs)/2

In [ ]:
#all_trigs[-10:]

In [ ]:
all_trigs[:10]

In [ ]:
#events[:10]
all_trigs[-10:]

In [ ]:
behav_files

In [ ]:
behav_files=pd.DataFrame()
sub_name='228'
pattern=sub_name+'_alpha_placeholders_matchingBrad_*_*_*_*_*_*.csv'
pattern2=sub_name+'_alpha_placeholders_matchingBrad_*_*_*_*_*.csv'
pattern3=sub_name+'_alpha_placeholders_matchingBrad_*_*_*_*_*_TRUNCATED_TO_MATCH_BDF.csv'
pattern4=sub_name+'_alpha_placeholders_matchingBrad_*_*_*_*_*_*_TRUNCATED_TO_MATCH_BDF.csv'
behav_files=pd.DataFrame()
for f in os.listdir(ROOT_behav):
    if fnmatch.fnmatch(f,pattern) or fnmatch.fnmatch(f,pattern2) or fnmatch.fnmatch(f,pattern3) or fnmatch.fnmatch(f,pattern4):
        print(f)
        behav_file=pd.read_csv(ROOT_behav+f,engine='python')
        behav_files=behav_files.append(behav_file,ignore_index=True)


In [ ]:
behav_files=behav_files.sort_values(['block','trialNum']).reset_index()

In [ ]:
behav_files.head()

In [ ]:
#eyes=pd.read_csv(ROOT_eyetrack+'sub_197_trialsCompiled_TRUNCATED_TO_MATCH.csv')
#adjusted_behav=pd.read_csv(ROOT_behav+'197_alpha_placeholders_matchingBrad_01_2019_Nov_12_1319_TRUNCATED_TO_MATCH_BDF.csv')

In [ ]:
eyes=pd.read_csv(ROOT_eyetrack+'sub_228_trialsCompiled_TRUNCATED_TO_MATCH.csv')

In [ ]:
len(eyes)

In [ ]:

eyes.head()

In [ ]:
appended=pd.concat([behav_files,eyes],ignore_index=False,axis=1)
#appended.to_csv('/data/backed_up/shared/AlphaStudy_data/placeholders/eyetracking_compiled_trials/sub_228_trialsCompiled_FOR_PROBE.csv')

In [ ]:
#behav_files.append(eyes,ignore_index=True,axis=1)
appended

In [ ]:
pd.read_csv('/data/backed_up/shared/AlphaStudy_data/placeholders/eyetracking_compiled_trials/sub_212_trialsCompiled_FOR_CUES.csv')